<h1 style="align: center;font-size: 18pt;">Preparation</h1>

<hr style="height:2.5px">

This tutorial shows the user how to properly use methods in the `Preparation class` to prepare input files for the next step, which is constructing the ensemble via the `BICePs` `Restraint class`. The data from this tutorial can be found from this work (DOI: [10.1002/jcc.23738](https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.23738)).

<hr style="height:2.5px">

In [1]:
import mdtraj as md
import numpy as np
import pandas as pd
import biceps

BICePs - Bayesian Inference of Conformational Populations, Version 2.0


In this tutorial, we have two experimental observables: (1) [J couplings](https://en.wikipedia.org/wiki/J-coupling) for small molecules and (2) [NMR nuclear Overhauser effect (NOE)](https://en.wikipedia.org/wiki/Nuclear_Overhauser_effect) (pairwise distances).
First we need to perform conformational clustering on our MD simulations data. In this case, 100 metastable states are clustered. Now we need to prepare prior knowledge we learned from computational simulations. Normally, we used potential energy for each metastable state. In the [original work](https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.23738), Zhou et al did Quantum Mechanical (QM) calculations to refine each state and B3LYP energy was used as priors in BICePs calculation. Instructions of QM calculations are beyond the scope of this tutorial. Alternatively, we can estimate the potential energy using U = -ln(P) where P is the normalized population for each state. We also have built-in functions ([toolbox](https://biceps.readthedocs.io/en/latest/api.html#toolbox)) to conduct this conversion. You can find tutorials using functions in `toolbox.py` [here](https://biceps.readthedocs.io/en/latest/tutorials/Tools/toolbox.html).   


Next, we need to compute pairwise distances and J coupling constants for each clustered state. 
To compute pairwise distances, we recommend to use [MDTraj](http://mdtraj.org) (`biceps.toolbox.compute_distances`) which is free to download. 
To compare simulated conformational ensembles to experimental NOE measurements, we normally computed $<r^{-6}>^{-1/6}$. <b>For convenience in this tutorial, we assume the cluster center of each state is representative enough and simply compute pairwise distances for the cluster center conformation. In practice, we still recommend users to compute <u>ensemble-averaged</u> distance.</b>

In [2]:
# Compute model_data for NOE
data_dir = "../../datasets/cineromycin_B/"
outdir = "NOE/"
states = biceps.toolbox.get_files(data_dir+"cineromycinB_pdbs/*")
nstates = len(states)
ind=data_dir+'atom_indice_noe.txt'
ind_noe = ind
biceps.toolbox.mkdir(outdir)
model_data_NOE = biceps.toolbox.compute_distances(states, ind, outdir)

Next, we need to convert computed distance to Pandas `DataFrame`. 

In [3]:
# Now using biceps Preparation submodule
model_data_NOE = str(outdir+"*.txt")
exp_data_NOE = data_dir+"noe_distance.txt"
preparation = biceps.Restraint.Preparation(nstates=nstates, top=states[0])
preparation.prep_noe(exp_data_NOE, model_data_NOE, indices=ind_noe, outdir=outdir, verbose=False)

Now, let's take a look at what's inside the newly generated files.

In [4]:
pd.read_pickle(outdir+'0.noe')

restraint_index  atom_index1  res1 atom_name1  atom_index2  res2  \
0                 1           23  UNK1         H3           25  UNK1   
1                 2           23  UNK1         H3           27  UNK1   
2                 3           25  UNK1         H5           27  UNK1   
3                 4           24  UNK1         H4           26  UNK1   
4                 5           25  UNK1         H5           26  UNK1   
5                 6           26  UNK1         H6           27  UNK1   
6                 7           27  UNK1         H7           32  UNK1   
7                 8           27  UNK1         H7           30  UNK1   
8                 9           27  UNK1         H7           31  UNK1   
9                10           28  UNK1         H8           38  UNK1   
10               10           28  UNK1         H8           39  UNK1   
11               10           28  UNK1         H8           40  UNK1   
12               10           29  UNK1         H9           38  UNK1   
13               10           29  UNK1         H9           39  UNK1   
14               10           29  UNK1         H9           40  UNK1   
15               11           25  UNK1         H5           41  UNK1   
16               11           25  UNK1         H5           42  UNK1   
17               11           25  UNK1         H5           43  UNK1   
18               12           26  UNK1         H6           44  UNK1   
19               12           26  UNK1         H6           45  UNK1   
20               12           26  UNK1         H6           46  UNK1   
21               13           33  UNK1        H13           38  UNK1   
22               13           33  UNK1        H13           39  UNK1   
23               13           33  UNK1        H13           40  UNK1   
24               14           35  UNK1        H15           38  UNK1   
25               14           35  UNK1        H15           39  UNK1   
26               14           35  UNK1        H15           40  UNK1   
27               14           36  UNK1        H16           38  UNK1   
28               14           36  UNK1        H16           39  UNK1   
29               14           36  UNK1        H16           40  UNK1   
30               14           37  UNK1        H17           38  UNK1   
31               14           37  UNK1        H17           39  UNK1   
32               14           37  UNK1        H17           40  UNK1   

   atom_name2  exp     model  comments  
0          H5  2.5  2.864866       NaN  
1          H7  2.5  4.400116       NaN  
2          H7  2.5  4.045793       NaN  
3          H6  2.5  3.481033       NaN  
4          H6  2.5  2.566673       NaN  
5          H7  2.5  3.560656       NaN  
6         H12  2.5  2.736875       NaN  
7         H10  2.5  3.743715       NaN  
8         H11  2.5  2.555209       NaN  
9         H18  2.5  4.403192       NaN  
10        H19  2.5  3.903093       NaN  
11        H20  2.5  5.182293       NaN  
12        H18  2.5  3.300441       NaN  
13        H19  2.5  2.385795       NaN  
14        H20  2.5  3.943905       NaN  
15        H21  2.5  3.807880       NaN  
16        H22  2.5  3.045646       NaN  
17        H23  2.5  2.279226       NaN  
18        H24  2.5  2.480295       NaN  
19        H25  2.5  2.813619       NaN  
20        H26  2.5  3.665993       NaN  
21        H18  2.5  3.812247       NaN  
22        H19  2.5  4.298023       NaN  
23        H20  2.5  3.774488       NaN  
24        H18  2.5  3.920761       NaN  
25        H19  2.5  5.119042       NaN  
26        H20  2.5  4.011077       NaN  
27        H18  2.5  2.496061       NaN  
28        H19  2.5  3.837284       NaN  
29        H20  2.5  2.601645       NaN  
30        H18  2.5  2.810059       NaN  
31        H19  2.5  4.223544       NaN  
32        H20  2.5  3.634443       NaN

Now let's move on to J couplings. Model predictions of coupling constants from dihedral angles θ were obtained from Karplus relations chosen depending on the relevant stereochemistry. 

Again, we need to convert computed J coupling constants to Pandas `DataFrame`. 

In [11]:
# Compute model_data forJ coupling
ind = np.load(data_dir+'ind.npy') # atom indices of J coupling constants

outdir = "J/"
biceps.toolbox.mkdir(outdir)

# Karplus relations for each dihedral angles 
karplus_key=np.loadtxt(data_dir+'Karplus.txt', dtype=str)
print('Karplus relations', karplus_key)
biceps.toolbox.compute_nonaa_scalar_coupling(states=data_dir+"cineromycinB_pdbs/*.fixed.pdb",
        index=ind, karplus_key=karplus_key, outdir=outdir)

Karplus relations ['Allylic' 'Karplus_HH' 'Karplus_HH' 'Karplus_HH' 'Karplus_HH'
 'Karplus_HH' 'Karplus_antiperiplanar_O' 'Karplus_antiperiplanar_O'
 'Karplus_antiperiplanar_O' 'Karplus_antiperiplanar_O']


In [5]:
indices = data_dir+'atom_indice_J.txt'
exp_data_J = data_dir+'exp_Jcoupling.txt'
model_data_J = data_dir+"J_coupling/*.txt"

# Now using biceps Preparation submodule
outdir = "J_NOE/"
biceps.toolbox.mkdir(outdir)
preparation = biceps.Restraint.Preparation(nstates=nstates, top=states[0])
preparation.prep_J(exp_data=exp_data_J, model_data=model_data_J, indices=indices, outdir=outdir, verbose=False)

Now, let's take a look at what's inside the newly generated files.

In [6]:
pd.read_pickle(outdir+'0.J')

restraint_index  atom_index1  res1 atom_name1  atom_index2  res2  \
0                1           25  UNK1         H5            9  UNK1   
1                2           30  UNK1        H10           14  UNK1   
2                3           31  UNK1        H11           14  UNK1   
3                4           32  UNK1        H12           15  UNK1   
4                4           32  UNK1        H12           15  UNK1   
5                4           32  UNK1        H12           15  UNK1   
6                5           32  UNK1        H12           15  UNK1   
7                6           33  UNK1        H13           16  UNK1   
8                6           33  UNK1        H13           16  UNK1   
9                6           33  UNK1        H13           16  UNK1   

  atom_name2  atom_index3  res3 atom_name3  atom_index4  res4 atom_name4  exp  \
0         C6           10  UNK1         C7           26  UNK1         H6  5.0   
1        C11           15  UNK1        C12           32  UNK1        H12  1.5   
2        C11           15  UNK1        C12           32  UNK1        H12  1.5   
3        C12           20  UNK1        C16           38  UNK1        H18  6.0   
4        C12           20  UNK1        C16           39  UNK1        H19  6.0   
5        C12           20  UNK1        C16           40  UNK1        H20  6.0   
6        C12           16  UNK1        C13           33  UNK1        H13  1.5   
7        C13           21  UNK1        C17           35  UNK1        H15  7.0   
8        C13           21  UNK1        C17           36  UNK1        H16  7.0   
9        C13           21  UNK1        C17           37  UNK1        H17  7.0   

       model  comments  
0   3.279128       NaN  
1  13.336870       NaN  
2   0.600845       NaN  
3  13.877749       NaN  
4   3.339372       NaN  
5   1.828765       NaN  
6   0.464738       NaN  
7   2.956672       NaN  
8  10.805575       NaN  
9   1.232210       NaN

<h1 style="align: center;font-size: 18pt;">Conclusion</h1>

In this tutorial, we briefly showed how to use [Preparation](https://biceps.readthedocs.io/en/latest/biceps.html#preparation) class to prepare input files for BICePs using precomputed experimental observables. Currently, BICePs supports the following restraints:
**NOE, J couplings, Chemical Shifts, Protection Factors and more to come...**

In [4]:
print(biceps.toolbox.list_possible_restraints())

['Restraint_cs', 'Restraint_J', 'Restraint_noe', 'Restraint_pf']


In the example above, we showed how to deal with NOE and J couplings (non-natural amino acids). 

<!--For J couplings for natural amino acids, please check this tutorial. -->

Chemical shifts can be computed using different algorithm. We recommend to use [Shiftx2](http://www.shiftx2.ca) which is also available in MDTraj library. **We offer methods in `biceps.toolbox` to easily compute model data (chemical shifts, J coupling, NOE distances).**

Protection factors is a special observables which asks for extra work. We provide a [full example](https://biceps.readthedocs.io/en/latest/examples/full_examples/apomyoglobin/apomyoglobin.html) that includes **HDX** (protection factor) restraints in biceps.

Now that the input files are ready, we can move on to [Restraint](https://biceps.readthedocs.io/en/latest/examples/Tutorials/Prep_Rest_Post_Ana/restraint.html), where we construct a conformational ensemble. 


<h6 style="align: justify;font-size: 12pt"># <span style="color:red;">NOTE</span>: The following cell is for pretty notebook rendering</h6>

In [9]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../../theme.css", "r").read()
    return HTML(styles)
css_styling()